In [ ]:
# Import required packages

from googleapiclient.discovery import build
import pandas as pd
from google.colab import files

In [ ]:
api_key = 'XXXXXXXXXXXXXXXXXXXXXXXXXXX'
channel_id = 'UCNU_lfiiWBdtULKOw6X0Dig'

channel_ids = ['UCNU_lfiiWBdtULKOw6X0Dig',  # Krish Naik
               'UC7cs8q-gJRlGwj4A8OmCmXg',  # Alex The Analyst
               'UC8LUT6Qn7MSvPQPM8ZJsW8g',  # LearnWidGiggs
               'UCnz-ZXXER4jOvuED5trXfEA',  # TechTFQ
              #  'UC8uU_wruBMHeeRma49dtZKA',  # Chandoo
               'UCh9nVJoWXmFb7sLApWGcLPQ',  # CodeBasics
               ]

# Create youtube service
youtube = build('youtube', 'v3', developerKey = api_key)

In [ ]:
# Function to get single channel details

# def get_channel_stats(youtube, channel_id):   # Pass Youtube Service and Channel Id as parameters

#   request = youtube.channels().list(
#       part = 'snippet,contentDetails,statistics',
#       id=channel_id)

#   response = request.execute()
#   data = dict(Channel_name = response['items'][0]['snippet']['title'],
#               Subscribers =  response['items'][0]['statistics']['subscriberCount'],
#               Views = response['items'][0]['statistics']['viewCount'],
#               Total_videos = response['items'][0]['statistics']['videoCount']
#               )

#   return data




In [ ]:
# get_channel_stats(youtube, channel_id)

## **Function to fetch channel details**

In [ ]:
# Function to get channel details

def get_channel_stats(youtube, channel_ids):   # Pass Youtube Service and Channel Ids as parameters
  all_data = []
  request = youtube.channels().list(
      part = 'snippet,contentDetails,statistics',
      id= ','.join(channel_ids) )

  response = request.execute()

  for i in range (len(response['items'])):
    data = dict(Channel_name = response['items'][i]['snippet']['title'],
                Subscribers =  response['items'][i]['statistics']['subscriberCount'],
                Views = response['items'][i]['statistics']['viewCount'],
                Total_videos = response['items'][i]['statistics']['videoCount'],
                Playlist_id = response['items'][i]['contentDetails']['relatedPlaylists']['uploads']
                )
    all_data.append(data)

  return all_data

In [ ]:
channel_statistics = get_channel_stats(youtube, channel_ids)

In [ ]:
type(channel_statistics)

list

In [ ]:
channel_statistics

[{'Channel_name': 'LearnWidGiggs',
  'Subscribers': '50200',
  'Views': '3555519',
  'Total_videos': '281',
  'Playlist_id': 'UU8LUT6Qn7MSvPQPM8ZJsW8g'},
 {'Channel_name': 'codebasics',
  'Subscribers': '1040000',
  'Views': '85735057',
  'Total_videos': '877',
  'Playlist_id': 'UUh9nVJoWXmFb7sLApWGcLPQ'},
 {'Channel_name': 'Alex The Analyst',
  'Subscribers': '794000',
  'Views': '35399365',
  'Total_videos': '308',
  'Playlist_id': 'UU7cs8q-gJRlGwj4A8OmCmXg'},
 {'Channel_name': 'techTFQ',
  'Subscribers': '303000',
  'Views': '16705727',
  'Total_videos': '136',
  'Playlist_id': 'UUnz-ZXXER4jOvuED5trXfEA'},
 {'Channel_name': 'Krish Naik',
  'Subscribers': '967000',
  'Views': '99266749',
  'Total_videos': '1914',
  'Playlist_id': 'UUNU_lfiiWBdtULKOw6X0Dig'}]

In [ ]:
channel_data =  pd.DataFrame(channel_statistics)

In [ ]:
channel_data

,Channel_name,Subscribers,Views,Total_videos,Playlist_id
0,Alex The Analyst,793000,35359675,308,UU7cs8q-gJRlGwj4A8OmCmXg
1,LearnWidGiggs,50000,3546443,281,UU8LUT6Qn7MSvPQPM8ZJsW8g
2,techTFQ,303000,16689680,136,UUnz-ZXXER4jOvuED5trXfEA
3,Krish Naik,967000,99266749,1915,UUNU_lfiiWBdtULKOw6X0Dig
4,codebasics,1030000,85655111,877,UUh9nVJoWXmFb7sLApWGcLPQ


In [ ]:
channel_data.to_csv('Channel_data.csv')

In [ ]:
channel_data.dtypes

Channel_name    object
Subscribers     object
Views           object
Total_videos    object
Playlist_id     object
dtype: object

In [ ]:
# Converting datatypes from object to numeric

channel_data['Subscribers'] = pd.to_numeric(channel_data['Subscribers'])
channel_data['Views'] = pd.to_numeric(channel_data['Views'])
channel_data['Total_videos'] = pd.to_numeric(channel_data['Total_videos'])

In [ ]:
channel_data.dtypes

Channel_name    object
Subscribers      int64
Views            int64
Total_videos     int64
Playlist_id     object
dtype: object

# **Function to get video ids**

In [ ]:
channel_data

,Channel_name,Subscribers,Views,Total_videos,Playlist_id
0,Alex The Analyst,793000,35359675,308,UU7cs8q-gJRlGwj4A8OmCmXg
1,LearnWidGiggs,50000,3546443,281,UU8LUT6Qn7MSvPQPM8ZJsW8g
2,techTFQ,303000,16689680,136,UUnz-ZXXER4jOvuED5trXfEA
3,Krish Naik,967000,99266749,1915,UUNU_lfiiWBdtULKOw6X0Dig
4,codebasics,1030000,85655111,877,UUh9nVJoWXmFb7sLApWGcLPQ


In [ ]:
playlist_id = channel_data.loc[channel_data['Channel_name']=='techTFQ', 'Playlist_id'].iloc[0]
playlist_id

'UUnz-ZXXER4jOvuED5trXfEA'

In [ ]:
def get_video_ids(youtube, playlist_id):  # playlist_id refers to uploads_id of a particular channel

  request = youtube.playlistItems().list(
      part = 'contentDetails',
      playlistId = playlist_id,
      maxResults = 50)

  response = request.execute()

  video_ids= []

  for i in range(len(response['items'])):
    video_ids.append(response['items'][i]['contentDetails']['videoId'])

  nextPageToken = response.get('nextPageToken')  # get() returns the value only if nextPageToken is present, else it returns None
  more_pages = True

  while more_pages:
    if nextPageToken is None:
      more_pages = False
    else:
      request = youtube.playlistItems().list(
                part = 'contentDetails',
                playlistId = playlist_id,
                maxResults = 50,
                pageToken = nextPageToken)

      response = request.execute()

      for i in range(len(response['items'])):
        video_ids.append(response['items'][i]['contentDetails']['videoId'])

      nextPageToken = response.get('nextPageToken')  # get() returns the value only if nextPageToken is present, else it returns None



  return video_ids


In [ ]:
video_ids= get_video_ids(youtube, playlist_id)


In [ ]:
video_ids

['4Z67GTTWuzc',
 'jXKGsMPk1Hg',
 'r2QbEueaFR8',
 'cdTzo1eZZ50',
 'cXpIWgBW1mU',
 'kLSWP5zJAN8',
 'WumXyCh8U4g',
 'vyTMJhHyEtg',
 'L0Gj14qUWUY',
 '-23QiFuwt4k',
 'xY2622VV7jM',
 'cpelgvGwHcA',
 'kJc1aK00vwg',
 'FJkIf4OmWFI',
 'Oz8uzN_7MuM',
 'F-gETvj-oN0',
 '60MJSk56MZs',
 'ka9kDqkITX4',
 'WBqTj-FYux8',
 '78DNY36XxQw',
 'KrUIQAcFptY',
 'LBWXjtLNauQ',
 'oU8fhP17ozk',
 'o5W-iAK21ws',
 'Xx09nRpwEtU',
 '0w5mx6lfx1Y',
 'dgIYeUAOzbM',
 'DKYg8JahHI0',
 'RjZFC6NVUMc',
 'w67I_aPKygE',
 'rM1BVoBke04',
 'FRzbOb3jdLg',
 'CVSfxxAun-4',
 'tY4POdOldQM',
 'J_da4WKsrjE',
 'AZ29DXaJ1Ts',
 'ZML_EJrBhnY',
 '6lI0q2DV8DY',
 'nzJJP-uDIZc',
 'BNmoCYk58mU',
 'W5Wvyc9Pass',
 'oedv5lR3w_g',
 'HiscSRv7zWk',
 'PLN28d0PoFU',
 'GxmrInUIMAE',
 'D0-Qy7yk5TQ',
 'GHtX0QXfi6g',
 '0QcEqFrhb_Y',
 'smztq8sRAhk',
 '7skZzocEU6c',
 'LZGaRcDxj8I',
 'rBPQ5fg_kiY',
 'hvwltYazuQo',
 'a-hFbr-4VQQ',
 'pMNcPLc9Z7c',
 'zZKb8FQRShs',
 'jS5_hjFgfzA',
 '6UAU79FNBjQ',
 'ZwFfiadQB3k',
 'Det4ZjBSe3M',
 'xUsY2jWQa1w',
 'ueOUSjdAZY8',
 '90iK6g

In [ ]:
len(video_ids)

136

## **Function to fetch video details**

In [ ]:
def get_video_details(youtube, video_ids):

  all_video_stats=[]
  for i in range(0, len(video_ids), 50):

    request = youtube.videos().list(
              part = 'snippet, statistics',
              id = ','.join(video_ids[i:i+50]))

    response = request.execute()



    for video in response['items']:


        video_stats = dict(Title = video['snippet']['title'],
                        Publish_date = video['snippet']['publishedAt'],
                        Views= video['statistics']['viewCount'],
                        Likes= video['statistics']['likeCount'],
                        Favorites= video['statistics']['favoriteCount'],
                        Comments= video['statistics']['commentCount'],

                           )



        all_video_stats.append(video_stats)

  return all_video_stats

In [ ]:
video_details = get_video_details(youtube, video_ids)

In [ ]:
video_data= pd.DataFrame(video_details)

In [ ]:
video_data

,Title,Publish_date,Views,Likes,Favorites,Comments
0,Super Interesting SQL Problem | Practice SQL Q...,2024-06-03T12:30:07Z,15994,648,0,43
1,REAL SQL Interview PROBLEM by Capgemini | Solv...,2024-05-22T12:30:08Z,46029,1664,0,123
2,Student Result Generator SQL Interview Query -...,2024-03-30T16:42:44Z,10090,326,0,71
3,Solving an INTERMEDIATE SQL Interview Query - ...,2024-03-29T17:31:29Z,6283,219,0,39
4,SPLIT & JOIN - SQL Interview Problem - 28 | #3...,2024-03-28T15:39:21Z,5676,178,0,42
...,...,...,...,...,...,...
131,How to install PostgreSQL on Mac OS | Install ...,2020-11-16T02:28:09Z,121931,1029,0,120
132,Learn What is Database | Types of Database | DBMS,2020-08-30T00:38:24Z,395874,6354,0,169
133,Do you need a Smartwatch,2020-07-12T15:32:36Z,11705,185,0,43
134,MacBook Pro 13 2020 One Week Later Review,2020-06-29T15:03:19Z,1522,73,0,20


In [ ]:
video_data.dtypes


Title           object
Publish_date    object
Views           object
Likes           object
Favorites       object
Comments        object
dtype: object

In [ ]:
# Extracting Hour and adding it as a column
video_data['Hour'] = pd.to_datetime(video_data['Publish_date']).dt.strftime('%H')

In [ ]:
video_data['Hour']

0      12
1      12
2      16
3      17
4      15
       ..
131    02
132    00
133    15
134    15
135    14
Name: Hour, Length: 136, dtype: object

In [ ]:
# Formatting datatypes of columns
video_data['Publish_date'] = pd.to_datetime(video_data['Publish_date']).dt.date
video_data['Views'] = pd.to_numeric(video_data['Views'])
video_data['Likes'] = pd.to_numeric(video_data['Likes'])
video_data['Comments'] = pd.to_numeric(video_data['Comments'])

In [ ]:
# Adding Hour column
# video_data['Hour'] = pd.to_datetime(video_data['Publish_date']).dt.hour

In [ ]:
# Adding Month column
video_data['Month'] = pd.to_datetime(video_data['Publish_date']).dt.strftime('%b')

In [ ]:
video_data

,Title,Publish_date,Views,Likes,Favorites,Comments,Hour,Month
0,Super Interesting SQL Problem | Practice SQL Q...,2024-06-03,15994,648,0,43,12,Jun
1,REAL SQL Interview PROBLEM by Capgemini | Solv...,2024-05-22,46029,1664,0,123,12,May
2,Student Result Generator SQL Interview Query -...,2024-03-30,10090,326,0,71,16,Mar
3,Solving an INTERMEDIATE SQL Interview Query - ...,2024-03-29,6283,219,0,39,17,Mar
4,SPLIT & JOIN - SQL Interview Problem - 28 | #3...,2024-03-28,5676,178,0,42,15,Mar
...,...,...,...,...,...,...,...,...
131,How to install PostgreSQL on Mac OS | Install ...,2020-11-16,121931,1029,0,120,02,Nov
132,Learn What is Database | Types of Database | DBMS,2020-08-30,395874,6354,0,169,00,Aug
133,Do you need a Smartwatch,2020-07-12,11705,185,0,43,15,Jul
134,MacBook Pro 13 2020 One Week Later Review,2020-06-29,1522,73,0,20,15,Jun


In [ ]:
# Adding Day of Week
video_data['Day of Week']= pd.to_datetime(video_data['Publish_date']).dt.dayofweek.map({0:'Mon', 1: 'Tue', 2: 'Wed', 3: 'Thu', 4: 'Fri', 5: 'Sat', 6: 'Sun'})

In [ ]:
video_data

,Title,Publish_date,Views,Likes,Favorites,Comments,Hour,Month,Day of Week
0,Super Interesting SQL Problem | Practice SQL Q...,2024-06-03,15994,648,0,43,12,Jun,Mon
1,REAL SQL Interview PROBLEM by Capgemini | Solv...,2024-05-22,46029,1664,0,123,12,May,Wed
2,Student Result Generator SQL Interview Query -...,2024-03-30,10090,326,0,71,16,Mar,Sat
3,Solving an INTERMEDIATE SQL Interview Query - ...,2024-03-29,6283,219,0,39,17,Mar,Fri
4,SPLIT & JOIN - SQL Interview Problem - 28 | #3...,2024-03-28,5676,178,0,42,15,Mar,Thu
...,...,...,...,...,...,...,...,...,...
131,How to install PostgreSQL on Mac OS | Install ...,2020-11-16,121931,1029,0,120,02,Nov,Mon
132,Learn What is Database | Types of Database | DBMS,2020-08-30,395874,6354,0,169,00,Aug,Sun
133,Do you need a Smartwatch,2020-07-12,11705,185,0,43,15,Jul,Sun
134,MacBook Pro 13 2020 One Week Later Review,2020-06-29,1522,73,0,20,15,Jun,Mon


In [ ]:
# Adding Year
video_data['Year']= pd.to_datetime(video_data['Publish_date']).dt.year

In [ ]:
video_data.head()

,Title,Publish_date,Views,Likes,Favorites,Comments,Hour,Month,Day of Week,Year
0,Super Interesting SQL Problem | Practice SQL Q...,2024-06-03,15994,648,0,43,12,Jun,Mon,2024
1,REAL SQL Interview PROBLEM by Capgemini | Solv...,2024-05-22,46029,1664,0,123,12,May,Wed,2024
2,Student Result Generator SQL Interview Query -...,2024-03-30,10090,326,0,71,16,Mar,Sat,2024
3,Solving an INTERMEDIATE SQL Interview Query - ...,2024-03-29,6283,219,0,39,17,Mar,Fri,2024
4,SPLIT & JOIN - SQL Interview Problem - 28 | #3...,2024-03-28,5676,178,0,42,15,Mar,Thu,2024


In [ ]:
# Reordering Columns
new_order = ['Title', 'Views', 'Likes', 'Comments', 'Favorites', 'Publish_date', 'Hour', 'Day of Week', 'Month', 'Year']

# Reorder DataFrame
video_data = video_data[new_order]

In [ ]:
video_data.head()

,Title,Views,Likes,Comments,Favorites,Publish_date,Hour,Day of Week,Month,Year
0,Super Interesting SQL Problem | Practice SQL Q...,15994,648,43,0,2024-06-03,12,Mon,Jun,2024
1,REAL SQL Interview PROBLEM by Capgemini | Solv...,46029,1664,123,0,2024-05-22,12,Wed,May,2024
2,Student Result Generator SQL Interview Query -...,10090,326,71,0,2024-03-30,16,Sat,Mar,2024
3,Solving an INTERMEDIATE SQL Interview Query - ...,6283,219,39,0,2024-03-29,17,Fri,Mar,2024
4,SPLIT & JOIN - SQL Interview Problem - 28 | #3...,5676,178,42,0,2024-03-28,15,Thu,Mar,2024


In [ ]:
# Saving Dataframe as CSV
video_data.to_csv('techTFQ.csv')